In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("https://www.dropbox.com/s/vz1l18qa9xxvhy3/demand_trend.csv?dl=1", index_col=0)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
q = df[ ['date', 'demand'] ]
q.plot(x='date', y='demand', figsize=(10,10))

In [ ]:
df['y'] = df.fillna( method='ffill' ).fillna( method='bfill' )['demand']

In [ ]:
df = df.reset_index().rename(columns={'index' : 'basic_trend'})

In [ ]:
df

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()
holidays = cal.holidays(start=df['date'].min(), end=df['date'].max())

df['holiday'] = df['date'].isin(holidays)

In [ ]:
df['holiday'] = df['holiday'].astype(int)

In [ ]:
df

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

In [ ]:
X = df.drop(['y', 'demand'], axis=1).set_index('date')
y = df['y']

In [ ]:
X

In [ ]:
y

In [ ]:
reg.fit(X,y)

In [ ]:
intercept, coef = reg.intercept_, reg.coef_

In [ ]:
intercept, coef

In [ ]:
no_hol = [intercept + coef[0]*n for n in range(0,len(df))]

yes_hol = [intercept + coef[0]*n + coef[1] for n in range(0,len(df))]

Z = pd.DataFrame(index=df['date'])
Z['no_hol'] = no_hol
Z['yes_hol'] = yes_hol

In [ ]:
Z.plot(figsize=(10,10))

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dtr = DecisionTreeRegressor(random_state=0)

In [ ]:
#too many settings!!
dtr.fit(X,y)

In [ ]:
bad_idea = dtr.predict(X)

In [ ]:
Z = pd.DataFrame(index=df['date'])
Z['bad_idea'] = bad_idea

In [ ]:
#hmmm, nope ;)
Z.plot(figsize=(10,10))

In [ ]:
from sklearn.metrics import r2_score

#nope
r2_score(Z.values, y)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

#at least 3 generally
tscv = TimeSeriesSplit(n_splits=3)

dtr_scores = []
lin_scores = []

In [ ]:
for train, test in tscv.split(X):
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]
    
    dtr.fit(X_train, y_train)
    reg.fit(X_train, y_train)
    
    dtr_y_pred = dtr.predict( X_test )
    lin_y_pred = reg.predict( X_test )

    dtr_scores.append(r2_score(dtr_y_pred, y_test))
    lin_scores.append(r2_score(lin_y_pred, y_test))

In [ ]:
#baseline is zero
dtr_scores

In [ ]:
lin_scores

In [ ]:
from sklearn.model_selection import ShuffleSplit

ss = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)

dtr_scores = []
lin_scores = []

In [ ]:
import numpy as np
df['pred'] = np.nan

for train_index, test_index in ss.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    dtr.fit(X_train, y_train)
    reg.fit(X_train, y_train)
    
    dtr_y_pred = dtr.predict( X_test )
    lin_y_pred = reg.predict( X_test )

    dtr_scores.append(r2_score(dtr_y_pred, y_test))
    lin_scores.append(r2_score(lin_y_pred, y_test))
    
    df.iloc[test_index,-1] = lin_y_pred

In [ ]:
dtr_scores

In [ ]:
lin_scores

In [ ]:
df